# WebIO IJulia Demo

This notebook demonstrates the usage of WebIO.jl inside Jupyter notebooks. All of the examples are taken directly from the [WebIO README](https://github.com/JuliaGizmos/WebIO.jl/blob/master/README.md), so feel free to check out the README for additional details. 

In [ ]:
using WebIO
using JSExpr: @js, @new, @var

# Getting things to display

Whenever a code cell returns a WebIO.Node object, IJulia will render it. For example,

In [ ]:
node(:div, "Hello, World")

# Composing content

Let's say you want to display the following HTML:

```html
<ul class="my-list">
    <li>get milk</li>
    <li>make a pie</li>
</ul>
```

You can create a nested Node object:

In [ ]:
node(:ul,
    node(:li, "get milk"),
    node(:li, "make a pie"), attributes=Dict(:class => "my-list"))

# Loading JavaScript dependencies

You can load dependencies by creating a Scope object and passing in `imports` argument.

In [ ]:
w = Scope(imports=["//cdnjs.cloudflare.com/ajax/libs/p5.js/0.5.11/p5.js"])

onimport(w, @js function (p5)
    function sketch(s)
        s.setup = () -> s.createCanvas(640, 480)

        s.draw = function ()
          if s.mouseIsPressed
            s.fill(0)
          else
            s.fill(255)
          end
          s.ellipse(s.mouseX, s.mouseY, 80, 80)
        end
    end
    @new p5(sketch, this.dom.querySelector("#container"))
end)

n = w(dom"div#container"())

# Sending values from JavaScript to Julia

Below is a scope which communicates with Julia. The following scope contains a button which sends a random number, generated in JavaScript, to Julia. We will print this number on the Julia side.

In [ ]:
function random_print_button()
    w = Scope()

    obs = Observable(w, "rand-value", 0.0)

    on(obs) do x
        println("JS sent $x")
    end

    w(
      dom"button"(
        "generate random",
        events=Dict("click"=>@js () -> $obs[] = Math.random()),
      ),
    )
end

In [ ]:
random_print_button()

## iframe encapsulation

We can also encapsulate a scope inside an `<iframe>`, which isolates it from the containing page's styling and layout: 

In [ ]:
iframe(random_print_button())

# Sending values from Julia to JavaScript

Here's a clock where the time is formatted and updated every second from Julia. We use the onjs handler and mutate the #clock DOM element to acheive this.

In [ ]:
using Dates
w = Scope()
obs = Observable(w, "clock-value", "")

timestr() = Dates.format(now(), "HH:MM:SS")

# update timestamp every second
@async while true
    sleep(1)
    obs[] = timestr()
end

# on every update to `obs`, replace the text content of #clock
onjs(obs, @js val -> begin
    @var clock = this.dom.querySelector("#clock")
    clock.textContent = val
end)

w(
  dom"div#clock"(
    timestr(),
  ),
)

For an even easier way to send values from Julia to JavaScript, we can simply rely on the fact that WebIO knows how to render `Observable`s directly to HTML. In this case WebIO will automatically construct a `Scope` and insert the relevant JavaScript to update the rendered content whenever the `Observable` changes value:

In [ ]:
clock_obs = Observable(timestr())
@async while true
    sleep(1)
    clock_obs[] = timestr()
end
clock_obs